#Modules Importation and preparation

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
WORK_DIR = "/content/drive/My Drive/Colab Notebooks/Writer_identification"
!pip uninstall -y opencv-python
!pip install -U opencv-contrib-python
% cd "/content/drive/My Drive/Colab Notebooks/Writer_identification"

Requirement already up-to-date: opencv-contrib-python in /usr/local/lib/python3.6/dist-packages (4.4.0.42)
/content/drive/My Drive/Colab Notebooks/Writer_identification


In [38]:
import Clusterer, Global_feature_exractors, Local_features_extractor, Norms, Image, PCA_reduction, Distances
from Dataset_loader import load_dataset_all, load_dataset_IAM
from testing import get_test_sample, accuracy_optimised
import json, os
from sklearn.neighbors import BallTree

Precise the key of the entry in config_train.json file

In [39]:
training_object = "training_09Sep"

In [40]:
with open("./config_train.json") as config_file:
  configuration = json.load(config_file)["Sift_Vlad_Pca"][training_object]

In [41]:
BASE_FOLDER = configuration["base_folder"]
if not os.path.exists(BASE_FOLDER):
  os.mkdir(BASE_FOLDER)

#Train Model

In [42]:
def new_cluster_centers(images, path_to_save, nb_clusters, max_no_improvement):
	hellinger_normalization = Norms.Norm.hellinger_normalization
	local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

	images_pre = [Image.Image(path_image, local_features_extractor, verbose=True) for path_image in images]

	clusterer = Clusterer.Clusterer.fit_new_trainig(images_pre, 
																									path_to_save, 
																									nb_clusters=nb_clusters, 
																									max_no_improvement=max_no_improvement)

In [43]:
def new_pca_vlad(images, path_to_save_pca, cluster_centers_path, n_components_pca):

	clusters_centers = Clusterer.Clusterer.fit_ancient_data(cluster_centers_path)
	vlad = Global_feature_exractors.VLAD(clusters_centers)

	hellinger_normalization = Norms.Norm.hellinger_normalization
	local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

	images_pre = [Image.Image(path_image, local_features_extractor, global_feature_extractor=vlad, verbose=True) for path_image in images]
	vlad_vectors = [image.global_descriptor for image in images_pre]

	PCA_reduction.PCA_reduction.create_new_pca_model(vlad_vectors, 
	                                                 path_to_save=path_to_save_pca, 
																									 n_components=n_components_pca)

In [44]:
_, images = load_dataset_all(WORK_DIR)

In [45]:
nb_images_treat = configuration["nb_images_treat"]
new_cluster_centers(images=images[:nb_images_treat],
                    path_to_save=BASE_FOLDER+configuration["path_cluster_centers"], 
                    nb_clusters=configuration["Number_clusters"], 
                    max_no_improvement=configuration["max_no_improvement"])

I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
Init 1/3 with method: k-means++
Inertia for init 1/3: 65.679913
Init 2/3 with method: k-means++
Inertia for init 2/3: 66.599134
Init 3/3 with method: k-means++
Inertia for init 3/3: 66.385428
Minibatch iteration 1/98400: mean batch inertia: 0.229030, ewa inertia: 0.229030 
Minibatch iteration 2/98400: mean batch inertia: 0.218336, ewa inertia: 0.229008 
Minibatch iteration 3/98400: mean batch inertia: 0.229707, ewa inertia: 0.229009 
Minibatch iteration 4/

In [46]:
new_pca_vlad(images=images[:nb_images_treat],
              path_to_save_pca=BASE_FOLDER+configuration["path_pca_model"], 
              cluster_centers_path=BASE_FOLDER+configuration["path_cluster_centers"], 
              n_components_pca=configuration["pca_number_components"])

I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !


#Test Model

In [47]:
import Dataset_loader
import importlib
importlib.reload(Global_feature_exractors)

<module 'Global_feature_exractors' from '/content/drive/My Drive/Colab Notebooks/Writer_identification/Global_feature_exractors.py'>

In [48]:
def test_sift_descr_vlad(images, writers, cluster_centers_path, accuracy_calculator, pca_path=None):
  
  hellinger_normalization = Norms.Norm.hellinger_normalization
  local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

  clusters_centers = Clusterer.Clusterer.fit_ancient_data(cluster_centers_path)
  if pca_path is not None: pca_instance = PCA_reduction.PCA_reduction(pca_path)
  else: pca_instance = None
  vlad = Global_feature_exractors.VLAD(clusters_centers, pca_instance=pca_instance)

  images_pre = [Image.Image(path_image, local_features_extractor, global_feature_extractor=vlad) for path_image in images]
  
  chi2_distance = Distances.Distance.chi2_distance
  
  accuracy_value = accuracy_calculator(np.array(images_pre), writers, vlad, chi2_distance)
  
  return accuracy_value

In [49]:
writers, images = load_dataset_IAM(WORK_DIR)
X_test, y_test = get_test_sample(images, writers, configuration["test_sample_size"])

In [50]:
accuracy_value = test_sift_descr_vlad(images=X_test,
                    writers=y_test,
                    cluster_centers_path=BASE_FOLDER+configuration["path_cluster_centers"],
                    accuracy_calculator = accuracy_optimised,
                    pca_path=BASE_FOLDER+configuration["path_pca_model"])

print()
print("Accuracy value for", training_object," :",accuracy_value*100,"%")

IAM_405  ==>  IAM_405 ( j04-046 , j04-035 )
IAM_414  ==>  IAM_405 ( j04-115 , j04-046 )
IAM_443  ==>  IAM_443 ( k04-054 , k04-050 )
IAM_443  ==>  IAM_443 ( k04-050 , k04-054 )
IAM_339  ==>  IAM_587 ( g06-018h , n04-209 )
IAM_213  ==>  IAM_213 ( e01-050 , e01-059 )
IAM_339  ==>  IAM_179 ( g06-011h , d01-085 )
IAM_192  ==>  IAM_192 ( d04-071 , d04-075 )
IAM_587  ==>  IAM_587 ( n04-202 , n04-209 )
IAM_405  ==>  IAM_405 ( j04-035 , j04-046 )
IAM_409  ==>  IAM_409 ( j04-076 , j04-070 )
IAM_179  ==>  IAM_179 ( d01-085 , d01-080 )
IAM_414  ==>  IAM_414 ( j04-111 , j04-115 )
IAM_447  ==>  IAM_447 ( k04-093 , k04-085 )
IAM_587  ==>  IAM_587 ( n04-209 , n04-202 )
IAM_179  ==>  IAM_179 ( d01-080 , d01-085 )
IAM_409  ==>  IAM_409 ( j04-070 , j04-076 )
IAM_447  ==>  IAM_447 ( k04-085 , k04-093 )
IAM_213  ==>  IAM_213 ( e01-059 , e01-050 )
IAM_192  ==>  IAM_192 ( d04-075 , d04-071 )

Accuracy value for training_09Sep  : 85.0 %
